# Examples of Plotting WRF and FV3 Forecast Files

FV3 forecasts are written out in two separate files at each output time. Files prefixed `dynf` contain 3D dynamics variables, while files prefixed `phyf` contain physics forecast information, mainly at the surface.

In [ ]:
from argparse import Namespace
import math
import os
import sys
import warnings
import yaml

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid
import numpy as np
from netCDF4 import Dataset
from wrf import getvar, interplevel, to_np, latlon_coords

warnings.filterwarnings('ignore') # Quiet the Jupyter Notebook reports of warnings in matplotlib/basemap

In [ ]:
# Provide a file path to a forecast directory. 
# The example below creates a dictionary containing 0hr forecast files for the HRRR and FV3 experiments


file_path = '/scratch1/BMC/wrfruc/chunhua/fv3sar-testing/data/2019091912.hrrr.arw/wrfprd.old'
fn_hrrr = os.path.join(file_path, 'wrfout_d01_2019-09-19_18_00_00')

file_path = '/scratch2/BMC/wrfruc/cholt/work/fv3sar_data/chunhua/2019091912.GSD'
datasets = {
    'HRRR': Dataset(fn_hrrr, 'r'),
    'GSDFV3': {fn:  Dataset(os.path.join(file_path, fn + f'{0:03d}.nc'), 'r') for fn in ['dynf', 'phyf']},
}

# Print out the available variables

In [ ]:
for v, info in datasets['HRRR'].variables.items():
    try:
        print(v, ':', info.description, info.shape, info.units)
    except:
        print(v, ':', info.name)    


# Plot subplots with experiments and diffs

In [ ]:
def eq_contours(data, data2=None):
    '''
    Returns a balanced set of contours for data that has negative values.
    Also returns default colorbar to use for balanced, vs all positive values.
    '''
    
    minval = np.amin(data) if data2 is None else min(np.amin(data), np.amin(data2))
    maxval = np.amax(data) if data2 is None else max(np.amax(data), np.amax(data2))
    
    minval = math.floor(minval) if abs(minval) > 1 else minval
    maxval = math.ceil(maxval) if maxval > 1 else maxval
    if minval == maxval:
        return np.linspace(-1, 1, 5), 'seismic'
    if np.amin(data) < 0:
        # Set balanced contours. Choose an odd number in linspace below
        maxval = max(abs(minval), abs(maxval))
        return np.linspace(-maxval, maxval, 21), 'seismic'
    else:
        return np.linspace(minval, maxval, 21), 'jet'

In [ ]:
def plot_data(data, lat, lon, title, expt, fig, ax, contours=None, cm=None):
    
    '''
    Input parameters:
    
        dataL: 2D Numpy array to be plotted in Left column
        dataR: 2D Numpy array to be plotted in Right column
        lat: 2D Numpy array of latitude
        lon: 2D Numpy array of longitude
        title: String describing the variable being plotted.
        
    Draws a Basemap representation with the contoured data overlayed, 
    with a colorbar for each experiment, and the difference between the two.
        
    '''
    
    def trim_grid():
        '''
        The u, v, and H data from analysis are all on grids either one column, or one row smaller than lat/lon. 
        Return the smaller lat, lon grids, given the shape of the data to be plotted.
        Has no effect when all grids are the same size.
        '''
        y, x = np.shape(data)
        return lat[:y, :x], lon[:y, :x]
                   
    
    lat_trim, lon_trim = trim_grid()

    # Check out this link for all cmap options: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
    # A good redwhiteblue cmap for increments is seismic, and for full fields with rainbow, change to jet

    m = Basemap(projection='mill', 
                llcrnrlon=lon.min()-2,
                urcrnrlon=lon.max()+2,
                llcrnrlat=lat.min()-2,
                urcrnrlat=lat.max()+2,
                resolution='l',
                ax=ax,
               )
    x, y = m(lon_trim, lat_trim)

    # Use the same contour values for both experiments.
    contours, cm = eq_contours(data) if contours is None else (contours, cm)
    
    # Draw the contoured data over the map
    cs = m.contourf(x, y, data, contours, cmap=cm, ax=ax)
    m.drawcoastlines();
    m.drawmapboundary();
    m.drawparallels(np.arange(-90.,120.,5),labels=[1,0,0,0]);
    m.drawmeridians(np.arange(-180.,180.,5),labels=[0,0,0,1]);
    fig.colorbar(cs, ax=ax, orientation='vertical', shrink=0.25);
    ax.set_title(f"{expt}: {title}")
    return contours, cm

# Load in the variable information

The mapping of variables between WRF and FV3 is not 1:1. The file `variable_mapping.yaml` contains information about atmospheric variables as they relate to each model. The `yaml` file is read into a dictionary in the following cell.

The keys in that file generally follow this heirarchy:
```
   generic variable name:  # Name to be referenced in this script
       description:        # Brief description of the variable
       fv3:                # Section containing FV3-specific settings
           file:           # Prefix of file that contains this variable [phyf, dynf]
           name:           # Name of NetCDF variable
       unit:               # Preferred unit
       wrf:                # Secction containing WRF-specific settings
           method:         # The retrieval method for the variable [direct, getvar] 
           name:           # The name of the variable to retrieve, if "direct" method, 
                           # the NetCDF variable, if "getvar" method, then the variable 
                           # name it expects
           
```

In [ ]:
with open('variable_mapping.yaml') as fn:
    var_map = yaml.load(fn, Loader=yaml.FullLoader)


# Plot the variables

## A little info about dictionary references

Referencing dictionary entries can be done with the following syntax:

```
     my_dict = {'a': 2, 'b': 3, 'd': {'apple': 'red', 'banana': 'orange'}}
     my_dict['a']            # returns 2
     my_dict['d']['apple']   # returns 'red'
     my_dict['c']            # Run time error!
```

To avoid the run time error when entries don't exist, we should make use of the dictionary attribute `get()`. By default, `get()` returns `None` if the key does not exist. If we have a nested dictionary, this can also be problematic for retrieving missing values. To ensure we move forward in the event we don't care about missing values we can use the following syntax:

```
    my_dict.get('c')                   # Returns None
    my_dict.get('c', {})               # Returns an empty dict
    my_dict.get('c', {}).get('apple')  # Returns None
```

In [ ]:
def to_fraction(arr):
    
    '''Given an array of values, returns the array in percent'''
    
    return arr / 100.0

In [ ]:
def fun_call_by_name(val):
    
    ''' Given an input string, val, returns the corresponding callable function.'''
    
    if '.' in val:
        module_name, fun_name = val.rsplit('.', 1)
        # you should restrict which modules may be loaded here
        assert module_name.startswith('my.')
    else:
        module_name = '__main__'
        fun_name = val
    try:
        __import__(module_name)
    except ImportError as exc:
        raise ConstructorError(
            "while constructing a Python object", mark,
            "cannot find module %r (%s)" % (utf8(module_name), exc), mark)
    module = sys.modules[module_name]
    fun = getattr(module, fun_name)
    return fun

In [ ]:
def transform(data, model, var, var_map):
    
    '''Applies a trasnformation on the input data given settings in var_map.'''
    
    # Check to see if a transformation is needed.
    transform = var_map.get(var, {}).get(model, {}).get('transform', False)
    
    if transform:
        f = fun_call_by_name(transform)
        return f(data)
    return data

In [ ]:
fhr = 0

# Load in the latitude/longitude variables from each model.
# Since these are different, we won't remap here, just plot on the NetCDF map provided.
lat_f = datasets['GSDFV3']['dynf']['grid_yt'][::] * 180 / math.pi
lon_f = datasets['GSDFV3']['dynf']['grid_xt'][::] * 180 / math.pi

lat_h = np.squeeze(datasets['HRRR']['XLAT'][::])
lon_h = np.squeeze(datasets['HRRR']['XLONG'][::])


# List of two experiments, and diff for labeling plots
expt = ['GSDFV3', 'HRRR']

# Variables to plot from dynf and phyf files
# To add additional variables, corresponding entries must be made in variable_mapping.yaml
vars_ = {
    'dynf': ['u', 'v', 'temp', 'specific_humidity', 'delz'],
    'phyf': ['temp_sfc', 'soil_temp', 'albedo_ave', 'ground_flux', 'soil_type', 'orog', 'pbl_height'],
}


# Vertical level from FV3 (surface is last index, -1; TOA is first index, 0)
lev_fv3 = -1
lev_wrf = 0


for file, varlist in vars_.items():
    nvar = len(varlist)

    # Make a figure with subplots for each variable (nvar rows)
    fig, ax = plt.subplots(nvar, 2, figsize=(24, 8*nvar))


    for i, var in enumerate(varlist):

        #### Get/plot data from FV3 ###
        
        # Ensure that "None" returns if the variable does not exist in the var_map
        fv3_var = var_map.get(var, {}).get('fv3', {}).get('name')
        titleL = f'{fv3_var} at {fhr} hr fcst'
        data = np.squeeze(datasets[expt[0]][file][fv3_var][::])
        
        # Transform data, if needed
        data = transform(data, 'fv3', var, var_map)
        dataL = data[lev_fv3] if data.ndim == 3 else data

        #### Get data from WRF ###
        wrf_var = var_map.get(var, {}).get('wrf', {}).get('name')
        titleR = f'{wrf_var} at {fhr} hr fcst'

        if wrf_var:
            data = getvar(datasets[expt[1]], wrf_var, squeeze=True)
            # Transform data, if needed
            data = transform(data, 'wrf', var, var_map)
                
        else:
            print(f'Plotting zeros for HRRR {var}')
            data = np.zeros_like(lat_h)

        dataR = data[lev_wrf] if data.ndim == 3 else data        
        
        # Determine contours based on both datasets and plot
        contours, cm = eq_contours(dataL, dataR)
        plot_data(dataL, lat_f, lon_f, titleL, expt[0], fig, ax[i, 0], contours, cm)
        plot_data(dataR, lat_h, lon_h, titleR, expt[1], fig, ax[i, 1], contours, cm)
